# Training and Sampling a Restricted Boltzmann Machine with QuCumber

Welcome to the tutorial for QuCumber, by PIQuIL.

<img src="rsz_1pickle_logo_white-1.png" width="300">

The purpose of this tutorial will be to walk you through what QuCumber can do. QuCumber is a neural network quantum state tomography (QST) package that can reconstruct a postitive-real and/or a complex wavefunction (in development) given your data as input and that can generate new data given trained parameters. We recommend that you read through our brief, but thorough, theoretical breakdown of Restricted Boltzmann Machines (RBM) - the neural network behind QuCumber - before going through with this tutorial if you do not have any previous knowledge of RBMs. **LINK NEEDED**

If you'd like to skip all of this and start using QuCumber on your machine with your own data, please click [here](./QuCumberTutorial_YourMachine.html).

## Download the Tutorial

If you'd like to run the tutorial on your machine, please head to **LINK NEEDED** and download the tutorial folder, which should contain the following.

 - rbm_tutorial.py,
 - cplx.py
 - unitary_library.py
 - observables.py
 - tfim1d_N10_train_samples.txt,
 - 2qubits_train_samples.txt,
 - tfim1d_N10_psi.txt,
 - 2qubits_train_bases.txt,
 - 2qubits_psi.txt,
 
## Comments Before Getting Started

This tutorial is sectioned into two parts that will walk you through the following.

1. Training an RBM to reconstruct a positive-real wavefunction (i.e. no phase) and then generating new data.
2. Training an RBM to reconstruct a complex wavefunction (i.e. with a phase) and then generating new data.

<p style="color:red"><i><b>Before you begin, please ensure that you have installed the following.</b></i></p>

 - Python 3
     - numpy, matplotlib, csv (should come with conda)
     - tqdm (a progress bar package: https://pypi.org/project/tqdm/3.1.4/)
 - Pytorch (https://pytorch.org/)
 
Go to:

[Part 1](./QuCumberTutorialReal.html)
[Part 2](./QuCumberTutorialComplex.html)